# Decorators
This tutorial introduces decorators. A decorator is a _wrapper_ that modifies the behavior of a function or class. The vast majority of use cases are to modify functions, rather than classes, so we will focus on that use case going forward.

# A decorator example--debugging instrumentation

The following example will help illustrate the kinds of situations in which we desire something like a decorator. Suppose we want to instrument some (or all) of our functions with debugging logic that tells us when the function has been entered and tells us when it has been exited. Consider the following pair of functions:

In [1]:
# Functions that construct greeting and parting sentences.

# Construct a greeting sentence
def make_greeting(name):
    return "Hello, " + name
    
# Construct a parting sentence.
def make_parting(name):
    return "Goodbye, " + name
    
# Run example
print(make_greeting('Josh'))
print(make_parting('Josh'))

Hello, Josh
Goodbye, Josh


## A solution with problems--inline the debugging logic
If we want to add debugging logic to these functions, we can change them to something like this.

In [2]:
# Functions that say "hello" and "goodbye" instrumented with debugging logic.

# Construct a greeting sentence
def make_greeting(name):
    print("Enter make_greeting, arg =", name)
    result = "Hello, " + name
    print(result, "<-- Exit make_greeting")
    
    return result
    
# Construct a parting sentence.
def make_parting(name):
    print("Enter make_greeting, arg =", name)
    result = "Goodbye, " + name
    print(result, "<-- Exit make_parting")
    
    return result
    
# Run example
print(make_greeting('Josh'))
print()
print(make_parting('Josh'))

Enter make_greeting, arg = Josh
Hello, Josh <-- Exit make_greeting
Hello, Josh

Enter make_greeting, arg = Josh
Goodbye, Josh <-- Exit make_parting
Goodbye, Josh


The above solution is sub-optimal for several reasons:

  - The code necessary to instrument the functions is the same in all cases. It is "boiler-plate" code, or template-code. To implement it for a new function, we have to remember the exact template, or we risk implementing it incorrectly.
  - If we decide in the future to change how the debugging works, we would have to make changes all over the code base.
  - The debugging code is orthogonal to the central purposes of the functions it modifies, obscuring these purposes and making the code harder to understand.
  
Ideally, we would like to isolate the code that handles the debugging logic in a single place and have it not be visible in the functions that use it. If we did so, this would immediately remove all three problems above.

This is the reason we desire decorators. A decorator is usually created because it represents a general-purpose modification that could be used on many different functions, irrespective of the central purpose of the function. Applying the decorator to the function endows the function with the enhancement associated with the decorator, and it does so without cluttering the code of the function or requiring the coder to remember the details required to implement a specific code template.

## An "old-school" solution--no decorators

As it turns out, we don't really need decorators to achieve all of the above in python. A _decorator_ is really just _syntactic sugar_ that makes it easier and more elegant to _wrap_ a function. Below we illustrate how to wrap a function "old school", without the use of a decorators.

In [3]:
# Define a function that implements the debugging logic. It will "wrap" the
# desired function.

def debug(f):
    """
    Return a wrapped function that implements debugging
    logic for the function f.
    """
    def wrapper(*args, **kwds):
        # prologue
        print("Enter", f.__name__, end = '')
        print(", args = ", args, sep = '', end = '')
        print(", kwds =", kwds)
        
        result = f(*args, **kwds) # Call the original function.
        
        # epilogue
        print(result, "<-- Exit", f.__name__)

        return result
    
    return wrapper


# Construct a greeting sentence
def make_greeting(name):
    return "Hello, " + name
make_greeting = debug(make_greeting) # Wrap make_greeting() with debugging logic

# Construct a parting sentence
def make_parting(name):
    return "Goodbye, " + name
make_parting  = debug(make_parting)  # Wrap make_parting() with debugging logic
 

# Run example
print(make_greeting('Josh'))
print()
print(make_parting('Josh'))

Enter make_greeting, args = ('Josh',), kwds = {}
Hello, Josh <-- Exit make_greeting
Hello, Josh

Enter make_parting, args = ('Josh',), kwds = {}
Goodbye, Josh <-- Exit make_parting
Goodbye, Josh


## An elegant solution--with decorators
The code above achieves everything we would like. The debugging logic is safely put in a single place, and it no longer clutters the definitions of `make_greeting()` or `make_parting()`. The only slight inelegance is that we have to bash the definitions of `make_greeting()` and `make_parting()` on lines 28 and 33 above. Python introduced decorators in order to make these lines unnecessary. Here is the same code again, using decorators.

In [4]:
# Apply debug as a decorator to add debugging logic.

# We omit the definition of debug(). 
# We don't need to alter a single line of its code, 
# as defined above on [3]:4.

# Construct a greeting sentence
@debug                    # Here we employ debug() as a decorator.
def make_greeting(name):  # This is the function definition that is decorated.
    return "Hello, " + name

# Construct a parting sentence
@debug                    
def make_parting(name):
    return "Goodbye, " + name
 

# Run example
print(make_greeting('Josh'))
print()
print(make_parting('Josh'))    

Enter make_greeting, args = ('Josh',), kwds = {}
Hello, Josh <-- Exit make_greeting
Hello, Josh

Enter make_parting, args = ('Josh',), kwds = {}
Goodbye, Josh <-- Exit make_parting
Goodbye, Josh


# Decorators Defined
A decorator is a function that takes a single argument, which must also be a function. When a decorator is called, it returns as its value yet another function that is a wrapped version of the input argument.

In the above code, on line 8, the presence of the `@` sign before `debug` tells python to treat `debug` as a decorator. As such, it expects `debug` to be a function that takes a single argument that is also a function. It also expects there to be a function definition directly beneath the decorator. The python interpeter evaluates the function definition on line 9, resulting in a function object, and then calls `debug` on this function object, assigning the return value of `debug` to the global `make_greeting`. In order words, `debug` gets fed in the original definition of `make_greeting` and returns a wrapped version of it, which wrapped version is the one actually assigned to the global `make_greeting`.

# Decorators are quite useful
It may seem as though it isn't worth learning about something as fancy as decorators in order to implement something as simple as the above debugging logic; however, keep in mind that we intentionally keep examples simple here so that they will be more understandable.

In reality, a decorator for debugging would have to handle exceptions and would also have to perform some kind of indenting to keep track of call levels. Below is an updated decorator that does these things. As one can see, it is well worth isolating all the debugging logic within a single function. Line 25 below is wrapped by a significant amount of prologue and epilogue code.

# A more realistic version of the debugging decorator

In [5]:
# Define a decorator that adds debugging logic to any function.

DebugCallLevel = -1 # Tracks how many nested calls have been made.
DEBUG_INDENT   = 4  # How many spaces to lead with for each call-level.


def debug(f):
    """
    This is a decorator. Enhance the function f
    so that it prints out debugging information
    about when it is called and what results it returns.
    """
    def wrapper(*args, **kwds):
        global DebugCallLevel
        
        # Make sure our debug level is properly reset when any errors are encountered.
        try:
            # Prologue
            DebugCallLevel += 1
            
            print(' '*4*DebugCallLevel, "Enter", f.__name__, end = '')
            print(", args = ", args, sep = '', end = '')
            print(", kwds =", kwds)
            
            result = f(*args, **kwds) # Here we call the original function.
            
        # Epilogue
        except Exception as e:
            print(' '*DEBUG_INDENT*DebugCallLevel, "Exit", f.__name__, "--> Error:", e)
            if DebugCallLevel > 0:
                raise e
        else:
            print(' '*DEBUG_INDENT*DebugCallLevel, "Exit", f.__name__, "-->", result)

            return result
        finally:
            DebugCallLevel -= 1
    
    return wrapper


# Calculate the factorial function recursively.
# Used as a test case for our debug decorator.
@debug
def factorial(n):
    if n <= 0:
        return 1.0
    else:
        return n * factorial(n - 1)


# Run example
print(factorial(3))
print()
factorial('hello')
    

 Enter factorial, args = (3,), kwds = {}
     Enter factorial, args = (2,), kwds = {}
         Enter factorial, args = (1,), kwds = {}
             Enter factorial, args = (0,), kwds = {}
             Exit factorial --> 1.0
         Exit factorial --> 1.0
     Exit factorial --> 2.0
 Exit factorial --> 6.0
6.0

 Enter factorial, args = ('hello',), kwds = {}
 Exit factorial --> Error: '<=' not supported between instances of 'str' and 'int'


# Decorator Constructors
The expression that follows the `@` sign doesn't have to be a function name. It can be a function application that evaluates to a decorator. This allows us to define functions that are not decorators in themselves, but which consruct decorators. Read the example below, which alters the `debug` function defined above so that it is no longer a decorator but becomes a decorator constructor.

In [6]:
# Define a decorator generator that adds debugging logic to any function.

# Only print out debugging information beneath
# a certain call-level. Once the call-level gets too deep,
# debugging information is no longer printed out.

DebugCallLevel = -1 # Tracks how many nested calls have been made.
DEBUG_INDENT   = 4  # How many spaces to lead with for each call-level.

def debug(max_call_levels):
    """
    This is a decorator generator.
    
    Return a decorator appropriate for wrapping functions with debug information.
    
    :param max_call_levels: an integer >= 1. The number of call levels to show
                            debug info for.
    """
    def decorator(f):
        """This is a dynamically generated decorator.
    
           Enhance the function f so that it prints out debugging information
           about when it is called and what results it returns, so long as the
           call-level is less than max_call_levels.
        """
        def wrapper(*args, **kwds):
            global DebugCallLevel

            # Make sure our debug level is properly reset when any errors are encountered.
            try:
                # Prologue
                DebugCallLevel += 1

                if DebugCallLevel < max_call_levels: 
                    print(' '*4*DebugCallLevel, "Enter", f.__name__, end = '')
                    print(", args = ", args, sep = '', end = '')
                    print(", kwds =", kwds)
                    
                    
                result = f(*args, **kwds) # (8) Here we call the main function.

            # Epilogue
            except Exception as e:
                if DebugCallLevel < max_call_levels:
                    print(' '*DEBUG_INDENT*DebugCallLevel, "Exit", f.__name__, "--> Error:", e)
                if DebugCallLevel > 0:
                    raise e
            else:
                if DebugCallLevel < max_call_levels:
                    print(' '*DEBUG_INDENT*DebugCallLevel, "Exit", f.__name__, "-->", result)

                return result
            finally:
                DebugCallLevel -= 1
        
        return wrapper
    return decorator


# Calculate the factorial function recursively.
# Used as a test case for our debug decorator.
@debug(4)
def factorial(n):
    if n <= 0:
        return 1.0
    else:
        return n * factorial(n - 1)


# Run example
print(factorial(8))
print()
factorial('hello')
    

 Enter factorial, args = (8,), kwds = {}
     Enter factorial, args = (7,), kwds = {}
         Enter factorial, args = (6,), kwds = {}
             Enter factorial, args = (5,), kwds = {}
             Exit factorial --> 120.0
         Exit factorial --> 720.0
     Exit factorial --> 5040.0
 Exit factorial --> 40320.0
40320.0

 Enter factorial, args = ('hello',), kwds = {}
 Exit factorial --> Error: '<=' not supported between instances of 'str' and 'int'


In the example above, one can see that there are now three nested levels of function definition instead of just two, as we had before. This can be a little bit mind-boggling, and it's easy to get lost among the nested levels. We'll take them one at a time.

The outermost function, `debug`, is no longer a decorator. Instead it is a decorator _constructor_, which means it must return a decorator when called. It does that, by returning the function named `decorator`. This function is a decorator, because it takes a function `f` as an argument and returns the function `wrapper` as its return value. The function `decorator` is really just the same as the function we used to call `debug`. But, by adding one more level of indirection, we can now pass in the `max_call_levels` variable to `debug` and have that imbedded in the closure that results when we return `wrapper`.